In [1]:
from pathlib import Path
from omnibelt import toposort, load_json, save_json, pformat, pformat_vars
from itertools import product, combinations
# import pyperclip
import random
from PIL import Image
from graphviz import Digraph
import matplotlib.pyplot as plt
import io
# import clipboard
import networkx as nx
from tqdm.notebook import tqdm
from tabulate import tabulate
import matplotlib.pyplot as plt
import seaborn as sns
from omniply import tool, ToolKit, Context

import numpy as np
import torch
from causalbenchmark.novo.simulation.models import Bernoulli, ConditionalBernoulli, BernoulliNetwork
from causalbenchmark.novo.seeding import NewsHeadlines, StatisticsPrompting, Story
# from causalbenchmark.novo.seeding.prompting import GraphInfo
from causalbenchmark.novo.templating import FixedTemplate, SimpleTemplater
# from causalbenchmark.novo.verbalization import misc
from causalbenchmark.novo import misc
from causalbenchmark.novo.verbalization.decision import Decision
from causalbenchmark.novo.templating import Template
from causalbenchmark.novo.verbalization import Verbalizer, MarginalVerbalization, ConditionalVerbalization
from causalbenchmark.novo.verbalization.variable import VariableVerbalization
from causalbenchmark.novo.verbalization.flavors import PrecisePercent

In [2]:
path = misc.data_root() / 'stories'
story_names = [p.stem for p in path.glob('*.json')]
print(story_names)

['Impact_of_Covid_on_Auto_Industry_Job_Market', 'NHS_Glasgow_Coronavirus_Outbreak_Impact_Study', 'Impact_of_Refurbished_Tech_on_Student_Performance_Milan']


In [3]:
use_old_loading = False
# story_names = [
# 	# 'old/test19', 'old/test20', 'old/test21',
# 	# 'Facebook-WallStreet_Trends_Post-Verdict1',
# 	# 'RoyalFashionTrends_Analysis2',
# 	# 'EU Gas Alliance and Inflation Trends',
# 	
# 	'Impact_of_Covid_on_Auto_Industry_Job_Market',
# 	
# ]
stories = []
for name in story_names:
	story = Story(story_root=misc.data_root() / 'stories', story_id=name)#.populate_defaults()
	stories.append(story)
	if use_old_loading:
		story['stats'] = {int(k): v for k, v in story['stats'].items()}
		story['verbs'] = {k: {int(k2): v2 for k2, v2 in v.items()} for k, v in story['verbs'].items()}
		for k, v in story['verbs'].items():
			for k2, v2 in v.items():
				if 'value' in v2:
					v2['value'] = {int(k3): v3 for k3, v3 in v2['value'].items()}
		story['questions']['questions'] = {int(k): v for k, v in story['questions']['questions'].items()}
len(stories)

In [4]:
def old_story_atoms(story):
	varverbs = {}
	for var, raw in story['verbs'].items():
		info = {'values': {0: {}, 1: {}}}
		
		info['variable'] = raw[1]['variable']
		info['subject'] = raw[2]['subject']
		info['domain'] = raw[4]['domain']
		
		for i in [0, 1]:
			info_val = info['values'][i]
			info_val['verb'] = raw[2]['value'][i] # goes with subject
			info_val['phrase'] = raw[3]['value'][i]
			info_val['descriptor'] = raw[4]['value'][i] # goes with domain
			info_val['event'] = raw[5]['value'][i]
			info_val['conditional'] = raw[6]['value'][i]
			info_val['interventional'] = raw[7]['value'][i]
			
		varverbs[var] = info
	
	story['sheet'] = varverbs
	return story

def story_atoms(story):
	base_keys = ['descriptor', 'subject', 'pronoun', 'preposition', 'domain']
	value_keys = ['predicate', 'nounclause', 'subclause', 'condition', 'action']
	verbs = []
	for node in story['nodes']:
		raw = story['verbs'][node['name']]
		info = {'values': {0: {}, 1: {}}}
		info.update({key: raw[key] for key in base_keys})
		for i in [0, 1]:
			info_val = info['values'][i]
			info_val.update({key: raw[f'{key}{i}'] for key in value_keys})
		# node['verbs'] = info
		verbs.append(info)
	story['verbs'] = verbs
for story in stories:
	(old_story_atoms if use_old_loading else story_atoms)(story)

In [5]:
story = random.choice(stories)
# rawverbs = story['verbs']
# rawvars = {node['name']: node for node in story['nodes']}
# next(iter(rawverbs.values()))

In [6]:
node = random.choice(list(story['nodes']))
varval = 0
node['name'], node['values'][varval]

('Government Economic Policy', 'No policy change')

In [7]:
def get_varverb(node, verb, varval, **static):
	varverb = VariableVerbalization().populate_defaults().populate_variable({**node, **static}, verb, varval)
	return varverb

def sample_text(ident=None, *other_tools, **static):
	if ident is None:
		ident = {}
	story_id = ident.setdefault('story_id', random.randint(0,len(stories)-1))
	story = stories[story_id]
	
	var_id = ident.setdefault('var_id', random.randint(0,len(story['nodes'])-1))
	varval = ident.setdefault('value_id', random.choice([0, 1]))
	
	varverb = get_varverb(story['nodes'][var_id], story['verbs'][var_id], varval, **static)

	conditions = []
	
	parent_ids = {key: val for key, val in sorted(ident.items()) if key.startswith('parent_id')}
	if len(parent_ids) or ident.setdefault('num_parents', random.choice([0, 1, 2])):
		parent_idx_options = [i for i, node in enumerate(story['nodes']) if i != var_id]
		if not len(parent_ids):
			num = ident.setdefault('num_parents', random.choice([0, 1, 2]))
			parent_ids = {f'parent_id{i}': pid for i, pid in enumerate(random.sample(parent_idx_options, k=num))}
		
		pvals = ident.setdefault('parent_vals', [random.randint(0, 1) for _ in range(len(parent_ids))])
		ident['parent_vals'] = tuple(pvals)
		conditions = [get_varverb(story['nodes'][pid], story['verbs'][pid], pval) 
					  for (key, pid), pval in zip(parent_ids.items(), pvals)]
		for k, pid in parent_ids.items():
			ident[k] = pid
	
	verbalizer = Verbalizer().populate_defaults().add_variable(varverb, *conditions)
	verbalizer.extend(other_tools)
	verbalizer.update(ident)
	return verbalizer

In [21]:
samples = []

prior = {
	'story_id': 2,
	'var_id': 5,
	'value_id': 0,
	
	# 'clause_id': 0,
	# 'position_id': 3,
	
	'num_parents': 0,
	# 'parent_id1': 0,
	# 'parent_vals': [1],
	# 'parent_id2': 1,
	
}

others = [PrecisePercent()]

for _ in range(30):
	ident = prior.copy()
	ctx = sample_text(ident, *others, mean=0.12)
	text = ctx['sentence']
	ident.update(ctx.identity())
	ctx['ident'] = ident
	samples.append(ctx)

cols = []
for row in samples:
	for key in row['ident']:
		if key not in cols:
			cols.append(key)

table = [(
	# i, len([k for k in row['ident'] if k not in prior]),
		  row['sentence'], 
		  # row['ctx']['event'], row['ctx']['event'], row['ctx']['phrase'], row['ctx']['subject'], row['ctx']['verb'],  
		  # *[row['ident'].get(key, '-') for key in cols]
		  ) for i, row in enumerate(samples)]
print(tabulate(table, headers=['i', 'dof', 'text', *cols]))# *list(map(str, range(len(cols))))]))

i
------------------------------------------------------------------------------
There is a 12% probability that a student struggles with technology.
Among 12% of students, a student's tech literacy is low.
With a certainty of 12%, a student struggles with technology.
Among 12% of students, there is a lack of technological proficiency.
Among 12% of students, there is a lack of technological proficiency.
There is a 12% chance that technological skills are lacking in a student.
12% of the time a student is not tech-savvy.
With a confidence of 12%, a student's tech literacy is low.
Among 12% of students, a student's tech literacy is low.
The likelihood that a student lacks technological expertise is 12%.
There is a 12% probability that a student lacks technological expertise.
There is a 12% probability that technological skills are lacking in a student.
With a certainty of 12%, a student is not tech-savvy.
There is a 12% chance that a student's tech literacy is low.
With a confidence of 1

In [11]:
sample = samples[0]
ident = sample['ident']
text = sample['sentence']

print(text)
print(ident)

There is a 38% probability that economic indicators are positive, because there is robust growth in the global economy.
{'story_id': 0, 'var_id': 0, 'value_id': 1, 'parent_id1': 0, 'parent_vals': [1], 'marginal_id': 0, 'chance_word_id': 0, 'antecedent_id': 0, 'cond_conjunction_id': 3, 'conditional_id': 1, 'clause_id': 2, 'subclause_id': 1, 'cond0_clause_id': 1, 'cond0_nounclause_id': 1}


In [ ]:

class ParentVerbalization(ToolKit):
	@tool('antecedent')
	def construct_antecedent(self, parents):
		pass


In [10]:
cond_templates = [
	['{antecedent}', '{clause}'],
	['{clause},', '{antecedent}'],
]

@tool('antecedent')
def construct_antecedent(parents):
	
	
	
	pass


In [ ]:
ident = {}
ctx = sample_text(ident, mean=0.57)



text = Template.detok(ctx['clause'], capitalize=True, sentence=True)
ident.update(ctx.identity())
print(text)
ident

In [33]:

selection = {}




story = random.choice(stories)
varname = random.choice(list(rawverbs.keys()))
varinfo = story['sheet'][varname]
varval = 0

ctx = ClauseVerbalization().populate_variable_info(info, mean=0.57).populate_default(precise_templates=precise_templates, position_templates=position_templates)

# ctx['clause_id'] = 3
ctx['position_id'] = 0

text = Template.detok(ctx['clause'], capitalize=True, sentence=True)
print(text)
ctx.identity()

The chance that this incident was scarcely covered by the media is 57%.


{'verb_id': 1, 'clause_id': 1, 'position_id': 0, 'chance_word_id': 1}

In [10]:
# ClauseVerbalization._precise_templates[0]

In [8]:

prob_words = ['probability', 'chance', 'likelihood']
chance_words = ['probability', 'chance', 'likelihood', 'odds']



In [18]:
def detok(tokens):
	toks = [', ' if tok == ',' else tok for tok in tokens]
	sentence = ' '.join(toks) + '.'
	return sentence.capitalize()

In [ ]:
# 1. One of the variables is {descriptor}.
# 2. {subject} [often/usually/sometimes/rarely/etc.] {predicate}.
# 3. There is a [number]% chance that {noun-clause}.
# 4. [In/For/With/etc.] [number]% of {domain}, there {phrase}.
# 5. Conditional sentence: {condition}, [some consequence].
# 6. Interventional: If {action}, [some effect].

In [19]:
# The man that she loved went home.
# She realized that he loves her.

NameError: name 'AbstractGadget' is not defined

In [ ]:
# 1. One of the variables is {variable}.
# 2. We estimate {subject} [often/usually/sometimes/rarely/etc.] {value}.
# 3. There is a [number]% chance that {value}.
# 4. [number]% of {domain} {value}.
# 5. [number]% of the time {value}.
# 6. Conditional sentence: {value}, [some consequence].
# 7. Interventional: If {value}, [some effect].